# Smart-G demo notebook , part 2</b>

This is an interactive document allowing to run Smart-G with python and visualize the results. <br>
*Tips*: cells can be executed with shift-enter. Tooltips can be obtained with shift-tab. More information [here](http://ipython.org/notebook.html) or in the help menu. [A table of content can also be added](https://github.com/minrk/ipython_extensions#table-of-contents).

In [ ]:
%pylab inline
# next 2 lines allow to automatically reload modules that have been changed externally
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from smartg import Smartg, reptran_merge
from smartg import RoughSurface, LambSurface, FlatSurface, Environment
from atmosphere import AtmAFGL, AeroOPAC, CloudOPAC, diff1
from water import IOP_Rw
from tools.tools import SpecInt, SpecInt2, Irr, ReadREPTRAN_bands, reduce_reptran
from tools.luts import LUT, MLUT, Idx, merge, read_lut_hdf, read_mlut
from tools.smartg_view import plot_polar, smartg_view, input_view, mdesc, transect_view
from ipywidgets import interact, interact_manual
import hapi_gpu as hapi
from matplotlib import colors
colors_ = list(six.iteritems(colors.cnames))
# Add the single letter colors.
for name, rgb in six.iteritems(colors.ColorConverter.colors):
    hex_ = colors.rgb2hex(rgb)
    colors_.append((name, hex_))

# Transform to hex color values.
hex_ = [color[1] for color in colors_]
# Get the rgb equivalent.
rgb = [colors.hex2color(color) for color in hex_]

# Split the hsv values to sort.
rr = [color[0] for color in rgb]
gg = [color[1] for color in rgb]
bb = [color[2] for color in rgb]

# Sort by hue, saturation and value.
ind = np.lexsort((rr, gg, bb))
sorted_colors = [colors_[i] for i in ind]

# Equivalence Theorem

In [ ]:
%%time
# Here we compare the TOA radiance simulated with
# plane parallel and spherical atmospheres, at 443 nm.
atm=AtmAFGL('afglt',grid=np.linspace(100.,0.,num=100), O3=400., NO2=False)
results = []
le={}
le.update(th=np.linspace(0.,89.,num=24)*np.pi/180)
le.update(phi=np.array([0.],dtype=np.float32)*np.pi/180)
for pp in [True, False]:
    S=Smartg(pp=pp, alt_move=True, double=True)
    for BEER in [0, 1]:
        # Absorption is computed with Beer Law (BEER=1) or Single scattering albedo (BEER=0)
        results.append(S.run(THVDEG=0., wl=550., NBPHOTONS=1e8, le=le, atm=atm, BEER=BEER))
    
SSA_PP = results[0]['I_up (TOA)']
ET_PP  = results[1]['I_up (TOA)']
SSA_SP = results[2]['I_up (TOA)']
ET_SP  = results[3]['I_up (TOA)']

In [ ]:
((ET_SP-ET_PP)/ET_SP*100.).sub()[0,:].plot(fmt='k-o',vmin=-2,vmax=2)
((SSA_SP-SSA_PP)/SSA_SP*100.).sub()[0,:].plot(fmt='r-x',vmin=-2,vmax=2)
figure()
((SSA_SP-ET_SP)/SSA_SP*100.).sub()[0,:].plot(fmt='r-o')
((SSA_PP-ET_PP)/SSA_PP*100.).sub()[0,:].plot(fmt='k-o',vmin=-1,vmax=1)

# ALIS

## Pure scattering

In [ ]:
S0= Smartg(alis=None       , double=True)
S = Smartg(alis={'nlow':-1}, double=True)

In [ ]:
%%time
dl  = 50.
lmin= 400.
lmax= 700.
wref= (lmax-lmin)/2 + lmin
NW  = int((lmax-lmin)/dl) + 1
wl  = np.linspace(lmin,lmax,num=NW)
NL=51
aer=AeroOPAC('maritime_clean',.1,550.)
comp=[aer]
#comp=[]
znew =np.linspace(100.,0.,num=NL)
pfwav=[550.]
pfwav=None
atm=AtmAFGL('afglmw', grid=znew, O3=0., NO2=False, comp=comp, pfwav=pfwav)

surf=RoughSurface(SUR=1)
surf=None
le={}
le.update(th=np.array([20.])*np.pi/180)
le.update(phi=np.array([0.])*np.pi/180)
NB=5e6
mle=   S.run(THVDEG=30., wl=wl, le=le, surf=surf, NBPHOTONS=NB  , atm=atm, progress=False)
m0le= S0.run(THVDEG=30., wl=wl, le=le, surf=surf, NBPHOTONS=NB*5, atm=atm, progress=False, BEER=1)
print mle.attrs['kernel time (s)']

In [ ]:
m0le['I_up (TOA)'].sub()[:,0,0].plot(fmt='-xr')
mle['I_up (TOA)'].sub()[:,0,0].plot(fmt='x-b')

figure()
((m0le['I_up (TOA)'].sub()[:,0,0]-mle['I_up (TOA)'].sub()[:,0,0])/(mle['I_up (TOA)'].sub()[:,0,0])).plot(fmt='o-r')

## HITRAN

In [ ]:
hapi.db_begin('data')

### O2

In [ ]:
%%time
dl= 0.001
lmin=759.
lmax=769.001

NW=int((lmax-lmin)/dl)+1
vmin=1e7/lmax
vmax=1e7/lmin
dv=(vmax-vmin)/NW
#vmin=12980. #cm-1
#vmax=13200.25 #cm-1
hapi.fetch('O2i1',7,1,vmin-100,vmax+100)
hapi.fetch('O2i2',7,2,vmin-100,vmax+100)
hapi.fetch('O2i3',7,3,vmin-100,vmax+100)
#dv=.25 #cm-1
NL=21
#NW= int((vmax-vmin)/dv)
print 'NW: ', NW
pmin=0.001
pmax=1012.
pnew=np.linspace(pmin,pmax, num=NL)
#znew=abs(-8. *np.log(pnew/pmax))
znew =np.linspace(100.,0.,num=NL)

atm=AtmAFGL('afglmw', grid=znew, O3=0., NO2=False, pfwav=[760.])

ao2=np.zeros((NW,NL),dtype=float)
cao2=np.zeros((NW,NL),dtype=float)
i=0
for p,t,o2,z in zip(atm.prof.P,atm.prof.T,atm.prof.dens_o2,atm.prof.z):
    nuo2,coefo2 = hapi.absorptionCoefficient_Voigt_gpu(SourceTables=['O2i1','O2i2','O2i3'],HITRAN_units=False,
            OmegaRange=[vmin,vmax],OmegaStep=dv,GammaL='gamma_self',
            Environment={'p':p/1013.,'T':t})
    ao2[:,i] = coefo2*1e5
    i=i+1
wl=1e7/nuo2                             

In [ ]:
atm=AtmAFGL('afglmw', grid=znew, O3=0., NO2=False, pfwav=[760.], prof_abs=LUT(ao2))
atm.calc(wl).describe()

In [ ]:
S3=Smartg(alis={'nlow':3}, double=True)
le={}
le.update(th=np.array([20., 70.])*np.pi/180)
le.update(phi=np.array([180.])*np.pi/180)

In [ ]:
m=S3.run(wl=wl, atm=atm, le=le, NBPHOTONS=1e6, progress=False)
print m.attrs['kernel time (s)']

In [ ]:
f=figure()
f.set_size_inches(12,3)
m['I_up (TOA)'].sub()[:,0,0].plot(fmt='-')
m['I_up (TOA)'].sub()[:,0,1].plot(fmt='g-')

## Jacobians

### Sensitivity to O3 column amount

In [ ]:
%%time
dl= 5.
lmin=500.
lmax=650.
NW=int((lmax-lmin)/dl)+1
print 'N Wavelength: ', NW
wl=np.linspace(lmin,lmax,num=NW) 
wls  = np.concatenate((wl,wl,wl))
_O3 = 300. # standard level in DU
O3l=[_O3*0.5, _O3, _O3*1.5] # ranges
NL=50.
znew =np.linspace(100.,0.,num=NL)
surf=RoughSurface(SUR=1)
le={}
le.update(th=np.array([20.])*np.pi/180)
le.update(phi=np.array([180.])*np.pi/180)
NB=1e7

# we build 3 profiles concatenated in the wavelength dimension, coresponding to 3 Ozone amounts
for i,O3 in enumerate(O3l):
    pro = AtmAFGL('afglmw', grid=znew, NO2=False, O3=O3).calc(wl)
    if i!=0 :      
        ab  = np.concatenate((ab , pro['tau_abs'].data), axis=0)
        ray = np.concatenate((ray, pro['tau_sca'].data), axis=0)
        
    else: 
        ab = pro['tau_abs'].data
        ray= pro['tau_sca'].data

atm = AtmAFGL('afglmw', grid=znew, prof_ray=ray, prof_abs=LUT(diff1(ab, axis=1)))
atm.calc(wls).describe()

S = Smartg(alis={'nlow':3}, double=True)

In [ ]:
surf=RoughSurface(SUR=1)
mle1=S.run(THVDEG=30., wl=wls, le=le, surf=surf,NBPHOTONS=5e7, atm=atm, progress=False)
print mle1.attrs['kernel time (s)']
mle2=S.run(THVDEG=30., wl=wls, le=le, surf=surf,NBPHOTONS=5e7, atm=atm, progress=False)

In [ ]:
Ip1=mle1['Q_up (TOA)']*mle1['Q_up (TOA)']+mle1['U_up (TOA)']*mle1['U_up (TOA)']
DoLP1=(Ip1.apply(np.sqrt)/mle1['I_up (TOA)'])*100.
Ip2=mle2['Q_up (TOA)']*mle2['Q_up (TOA)']+mle2['U_up (TOA)']*mle2['U_up (TOA)']
DoLP2=(Ip2.apply(np.sqrt)/mle2['I_up (TOA)'])*100.
DoLP1.sub()[:,0,0].plot(fmt='b.')
figure()
diff1 =DoLP1.sub()[:,0,0]/DoLP1.sub()[0,0,0]
diff2 =DoLP2.sub()[:,0,0]/DoLP2.sub()[0,0,0]
diff1.plot(fmt='b.')
diff2.plot(fmt='rx')
figure()
(diff1/diff2).plot(fmt='kx')

### Sensitivity of O2A to surface pressure

In [ ]:
%%time
dl= 0.0100
lmin=759.
lmax=769.001
NW=int((lmax-lmin)/dl)
vmin=1e7/lmax
vmax=1e7/lmin
dv=(vmax-vmin)/NW
#vmin=12980. #cm-1
#vmax=13200.25 #cm-1
hapi.fetch('O2i1',7,1,vmin-100,vmax+100)
hapi.fetch('O2i2',7,2,vmin-100,vmax+100)
hapi.fetch('O2i3',7,3,vmin-100,vmax+100)
#dv=.25 #cm-1
NL=21
#NW= int((vmax-vmin)/dv)
print 'NW: ', NW
pmin=0.001
pmax=1012.
pnew=np.linspace(pmin,pmax, num=NL)
#znew=abs(-8. *np.log(pnew/pmax))
znew =np.linspace(100.,0.,num=NL)

NB=1e7

P0l=[950., 1010.]

ao2=np.zeros((NW,NL),dtype=float)

for i,P0 in enumerate(P0l):
    j=0
    atm = AtmAFGL('afglmw', grid=znew, NO2=False, O3=0., P0=P0)
    
    for p,t,o2,z in zip(atm.prof.P,atm.prof.T,atm.prof.dens_o2,atm.prof.z):
        nuo2,coefo2 = hapi.absorptionCoefficient_Voigt_gpu(SourceTables=['O2i1','O2i2','O2i3'],HITRAN_units=False,
            OmegaRange=[vmin,vmax],OmegaStep=dv,GammaL='gamma_self',
            Environment={'p':p/1013.,'T':t})
        ao2[:,j] = coefo2*1e5
        j=j+1
    wl=(1e7/nuo2)
    wl=wl[::1]

    pro = AtmAFGL('afglmw', grid=znew, NO2=False, O3=0., P0=P0).calc(wl)
    if i!=0 :      
        ab  = np.concatenate((ab , ao2), axis=0)
        ray = np.concatenate((ray, pro['tau_r'].data), axis=0)
        
    else: 
        ab = ao2
        ray= pro['tau_r'].data

wls = np.concatenate((wl,wl))
print ao2.max()
atm = AtmAFGL('afglmw', grid=znew, prof_ray=ray, prof_abs=LUT(ab))
#atm.calc(wls).describe()


In [ ]:
S0=Smartg(alis={'nlow':3}, double=True)
tv=np.linspace(0.,75.,num=6)
le={}
le.update(th=tv*np.pi/180)
le.update(phi=np.linspace(0.,360.,num=1)*np.pi/180)
surf=RoughSurface(SUR=1)
surf=None

In [ ]:
%%time
m=S0.run(THVDEG=30., wl=wls, atm=atm, surf=surf, le=le, NBPHOTONS=1e5, progress=False)

In [ ]:
field='I_up (TOA)'
I=m[field].sub(d={'wavelength':arange(NW)})
I.names[0]=r'$\lambda (nm)$'
I.axes[0]=wl
I.desc=mdesc(field)
for t in tv[:-1]:
    _= I.sub()[:,0,Idx(t)].plot(fmt='-', vmax=0.02, vmin=0, label='%.0f'%t)
#legend()

figure()
jac= (m[field].sub(d={'wavelength':arange(NW)+NW}) - 
      m[field].sub(d={'wavelength':arange(NW)}) ) / (P0l[-1]-P0l[0]) 
jac.names[0]=r'$\lambda (nm)$'
jac.axes[0]=wl
jac.desc=r'$d$'+mdesc(field)+r'$/dP_0$' + '  ' + r'$(hPa^{-1})$'

for t in tv[:-1]:
    _=jac.sub()[:,0,Idx(t)].plot(fmt='-', vmax=2e-5, vmin=0, label='%.0f'%t)
legend()

figure()
field='Q_up (TOA)'
I=m[field].sub(d={'wavelength':arange(NW)})
I.names[0]=r'$\lambda (nm)$'
I.axes[0]=wl
I.desc=mdesc(field)
for t in tv[:-1]:
    _= I.sub()[:,0,Idx(t)].plot(fmt='-', vmax=0.001, vmin=-0.003, label='%.0f'%t)
#legend()

figure()
jac= (m[field].sub(d={'wavelength':arange(NW)+NW}) - 
      m[field].sub(d={'wavelength':arange(NW)}) ) / (P0l[-1]-P0l[0]) 
jac.names[0]=r'$\lambda (nm)$'
jac.axes[0]=wl
jac.desc=r'$d$'+mdesc(field)+r'$/dP_0$' + '  ' + r'$(hPa^{-1})$'

for t in tv[:-1]:
    _=jac.sub()[:,0,Idx(t)].plot(fmt='-', vmax=0.5e-6, vmin=-2.5e-6, label='%.0f'%t)
#legend(loc='best')

### PARASOL O2

In [ ]:
thv = np.linspace(0., 60. , num=2)
phi = np.linspace(0., 180., num=1)
#thv = np.linspace(0., 60. , num=8)
#phi = np.linspace(0., 180., num=2)
PARASOL_SOLAR = REPTRAN('reptran_solar_parasol')
ibands = PARASOL_SOLAR.to_smartg(lmin=630, lmax=830)
NL = 25; wref = 700.
znew =np.linspace(100.,0.,num=NL)
S = Smartg(alis={'nlow':-1}, double=True)
S0 = Smartg(double=True)
le={}
le.update(th=thv*np.pi/180)
le.update(phi=phi*np.pi/180)

In [ ]:
delta = 0.5
zmin  = 1.
alb=0.1
mult  = np.array([1., 1+delta])
th0   = 30.
COT_l   = 10. * mult
#COT_l   = np.array([1.])
CGT_l   = 2.  * mult
#CGT_l   = np.array([2.])
#REFF_l  = 5.  * mult
REFF_l  = np.array([5.])

surf  = LambSurface(ALB=alb)
atms  = Profile('afglmw', grid=znew, pfwav=[wref], force_cache=True)
for COT in COT_l:
    for CGT in CGT_l:
        for REFF in REFF_l:
            ZMAX = zmin + CGT
            cloud = CloudOPAC('CUMA',[('wc.sol.mie', zmin, ZMAX , 1., REFF)], COT, wref)
            aer = AeroOPAC('desert', COT, wref)
            atm = Profile('afglmw', grid=znew, cloud=cloud, pfwav=[wref], force_cache=False)                           
            #atms.cache_prof_keys.append(ibands.l)
            tmp,_ = atm.calc_bands(ibands.l)
            atms.cache_prof_values.append(tmp)
                
NSIM = len(COT_l)*len(CGT_l)*len(REFF_l)
wl = ibands.l * NSIM    

In [ ]:
m= S.run(THVDEG=th0, wl=wl, NBPHOTONS=1e5, le=le, atm=atm, surf=surf)
print 'time :                            %s'%m.attrs['kernel time (s)']

In [ ]:
#m.describe(show_attrs=True)
print m['tauaer'][:,-1]

In [ ]:
NW=len(ibands.l)
for i in range(NSIM):
    s=reduce_reptran(m.sub(d={'Wavelength':np.arange(i*NW, (i+1)*NW)}), ibands)
    if i==0:
        f1=transect_view(s.sub(d={'Wavelength':0}), field='up (TOA)')
        f2=transect_view(s.sub(d={'Wavelength':1}), field='up (TOA)')
        f3=transect_view(s.sub(d={'Wavelength':2}), field='up (TOA)')
    else:
        _=transect_view(s.sub(d={'Wavelength':0}), field='up (TOA)', fig=f1)
        _=transect_view(s.sub(d={'Wavelength':1}), field='up (TOA)', fig=f2)
        _=transect_view(s.sub(d={'Wavelength':2}), field='up (TOA)', fig=f3)

## Emde et al 2010

### Rayleigh  and black surface

In [ ]:
S=Smartg(alis={'nlow':3}, double=True)

In [ ]:
fgas = '/home/mathieu/work/MCCuda/artdeco_output/cTauGas_ray_O2.dat'
gas_valid = diff1(np.loadtxt(fgas, skiprows=7)[:,1:].T, axis=1)
z_valid   = np.loadtxt(fgas, skiprows=7)[:,0]
w_valid   = np.array(open(fgas).readlines()[5].split()).astype(float)
fray = '/home/mathieu/work/MCCuda/artdeco_output/cTauRay_ray_O2.dat'
ray_valid = diff1(np.loadtxt(fray, skiprows=7)[:,1:].T, axis=1)
data_valid=np.loadtxt('/home/mathieu/work/MCCuda/resultat/artdeco_lbl_nstr_32_ray_O2.dat')

In [ ]:
atm_valid = AtmAFGL('afglmw', grid=z_valid, O3=0., NO2=False, pfwav=[760.], 
                    prof_ray= ray_valid, prof_abs= gas_valid)

In [ ]:
le={}
le.update(th=np.array([20.])*np.pi/180)
le.update(phi=np.array([180.])*np.pi/180)
mle_valid5=  S.run(THVDEG=30., wl=w_valid, le=le,  DEPO=0., NBPHOTONS=1e9, atm=atm_valid, 
                   NBLOOP=10000, progress=False)
print mle_valid5.attrs['kernel time (s)']

In [ ]:
i_valid=data_valid[:,1]
ip_valid=np.sqrt(data_valid[:,2]*data_valid[:,2]+data_valid[:,3]*data_valid[:,3])
ip=mle_valid5['Q_up (TOA)']*mle_valid5['Q_up (TOA)']+mle_valid5['U_up (TOA)']*mle_valid5['U_up (TOA)']
ip=ip.apply(np.sqrt,mdesc('Ip_up (TOA)'))
i=mle_valid5['I_up (TOA)']
##
plot(w_valid,  i_valid, 'r')
f= i.sub()[:,0,0].plot(fmt='g-', vmax=0.01)
figure()
df=i.sub()[:,0,0]- i_valid
dff=df/i_valid*100
dff.plot(fmt='k-', vmax=3, vmin=-3)
##
figure()
plot(w_valid,  ip_valid, 'r')
f= ip.sub()[:,0,0].plot(fmt='g-', vmax=0.005)
figure()
df=ip.sub()[:,0,0]- ip_valid
dff=df/ip_valid*100
dff.plot(fmt='k-', vmax=3, vmin=-3)

### Rayleigh , aerosol and black surface

In [ ]:
S=Smartg(alis={'nlow':3}, double=True)

In [ ]:
fgas = '/home/mathieu/work/MCCuda/artdeco_output/cTauGas_ray_desert_O2.dat'
gas_valid = diff1(np.loadtxt(fgas, skiprows=7)[:,1:].T, axis=1)
z_valid   = np.loadtxt(fgas, skiprows=7)[:,0]
w_valid   = np.array(open(fgas).readlines()[5].split()).astype(float)
fray = '/home/mathieu/work/MCCuda/artdeco_output/cTauRay_ray_desert_O2.dat'
ray_valid = diff1(np.loadtxt(fray, skiprows=7)[:,1:].T, axis=1)
faer_abs = '/home/mathieu/work/MCCuda/artdeco_output/cTauAbs_ptcle_ray_desert_O2.dat'
aer_abs_valid = diff1(np.loadtxt(faer_abs, skiprows=7)[:,1:].T, axis=1)
faer_sca = '/home/mathieu/work/MCCuda/artdeco_output/cTauSca_ptcle_ray_desert_O2.dat'
aer_sca_valid = diff1(np.loadtxt(faer_sca, skiprows=7)[:,1:].T, axis=1)

# aerosols phase matrix import
faer_phase= '/home/mathieu/work/MCCuda/artdeco_output/phasemat_ray_desert_O2.dat'
f=open(faer_phase,'r')
N=np.genfromtxt(faer_phase, usecols=range(1), max_rows=1, dtype=int)
pfwav=[]
P=[]
Npf=3
data=np.zeros((Npf,1,N,5), dtype=float32) 
for k in range(Npf):
    pfwav.append(np.genfromtxt(faer_phase, usecols=range(1), skip_header=(1+(2+N)*k), max_rows=1))
    pizero=np.genfromtxt(faer_phase, usecols=range(1), skip_header=(1+(2+N)*k+1), max_rows=1)
    data[k,0,:,:] = np.genfromtxt(faer_phase, usecols=range(5), skip_header=(1+(2+N)*k+2), max_rows=N)
    
data=data.swapaxes(2,3)
phase_valid = LUT(data[:,:,1:,:],
          names = ['wav_phase_atm', 'z_phase_atm', 'stk','theta'] ,
          axes  = [pfwav, [0], None, data[0,0,0,:]])
   
data_valid=np.loadtxt('/home/mathieu/work/MCCuda/resultat/artdeco_lbl_nstr_32_ray_desert_O2.dat')

In [ ]:
aer_ext_valid  = aer_sca_valid + aer_abs_valid
aer_ssa_valid  = aer_sca_valid / aer_ext_valid
aer_ssa_valid[aer_ext_valid==0]=1.
comp=[AeroOPAC('desert',0.5, 550., phase= phase_valid)]
atm_valid = AtmAFGL('afglmw', grid=z_valid, O3=0., NO2=False, pfwav=pfwav, comp=comp,
                    prof_ray= ray_valid,
                    prof_aer= (aer_ext_valid,aer_ssa_valid),
                    prof_abs= gas_valid
                    )

In [ ]:
le={}
le.update(th=np.array([20.])*np.pi/180)
le.update(phi=np.array([180.])*np.pi/180)
mle_valid5=  S.run(THVDEG=30., wl=w_valid, le=le,  DEPO=0., NBPHOTONS=1e8, atm=atm_valid, 
                   NBLOOP=10000, progress=False)
print mle_valid5.attrs['kernel time (s)']

In [ ]:
i_valid=data_valid[:,1]
ip_valid=np.sqrt(data_valid[:,2]*data_valid[:,2]+data_valid[:,3]*data_valid[:,3])
ip=mle_valid5['Q_up (TOA)']*mle_valid5['Q_up (TOA)']+mle_valid5['U_up (TOA)']*mle_valid5['U_up (TOA)']
ip=ip.apply(np.sqrt,mdesc('Ip_up (TOA)'))
i=mle_valid5['I_up (TOA)']
##
plot(w_valid,  i_valid, 'r')
f= i.sub()[:,0,0].plot(fmt='g-', vmax=0.01)
figure()
df=i.sub()[:,0,0]- i_valid
dff=df/i_valid*100
dff.plot(fmt='k-', vmax=10, vmin=-6)
##
figure()
plot(w_valid,  ip_valid, 'r')
f= ip.sub()[:,0,0].plot(fmt='g-', vmax=0.005)
figure()
df=ip.sub()[:,0,0]- ip_valid
dff=df/ip_valid*100
dff.plot(fmt='k-', vmax=10, vmin=-10)

## Tanso FTS

In [ ]:
S=Smartg(alis={'nlow':3}, double=True)

In [ ]:
fgas = '/home/mathieu/work/MCCuda/artdeco_output/cTauGas_ray_O2_fts1.dat'
gas_valid = np.loadtxt(fgas, skiprows=7)[:,1:].T
z_valid   = np.loadtxt(fgas, skiprows=7)[:,0]
w_valid   = np.array(open(fgas).readlines()[5].split()).astype(float)
fray = '/home/mathieu/work/MCCuda/artdeco_output/cTauRay_ray_O2_fts1.dat'
ray_valid = np.loadtxt(fray, skiprows=7)[:,1:].T
#data_valid=np.loadtxt('/home/mathieu/work/MCCuda/resultat/artdeco_lbl_nstr_32_ray_glitter_O2_fts1.dat')
data_valid=np.loadtxt('/home/mathieu/work/MCCuda/resultat/artdeco_lbl_nstr_32_ray_O2_fts1.dat')

In [ ]:
gg=LUT(diff1(gas_valid,axis=1))
atm_valid = AtmAFGL('afglmw', grid=z_valid, O3=0., NO2=False, pfwav=[760.], prof_ray= ray_valid, 
                    prof_abs=gg)

In [ ]:
##%prun  
le={}
le.update(th=np.array([20.])*np.pi/180)
le.update(phi=np.array([180.])*np.pi/180)
surf=RoughSurface(SUR=1)
surf=None
mle_valid_fts1=  S.run(THVDEG=30., wl=w_valid, le=le,  DEPO=0., NBPHOTONS=1e7, 
                       surf=surf, atm=atm_valid, NBLOOP=5000)
print mle_valid_fts1.attrs['kernel time (s)']

In [ ]:
f=figure()
f.set_size_inches(22,8)
mle_valid_fts1['Q_up (TOA)'].sub()[:,0,0].plot(fmt='b-')
mle_valid_fts1['I_up (TOA)'].sub()[:,0,0].plot(fmt='r-', vmax=0.02, vmin=-0.01)

In [ ]:
i_valid=data_valid[:,1]
ip_valid=np.sqrt(data_valid[:,2]*data_valid[:,2]+data_valid[:,3]*data_valid[:,3])
ip=mle_valid_fts1['Q_up (TOA)']*mle_valid_fts1['Q_up (TOA)']+mle_valid_fts1['U_up (TOA)']*mle_valid_fts1['U_up (TOA)']
ip=ip.apply(np.sqrt,mdesc('Ip_up (TOA)'))

i=mle_valid_fts1['I_up (TOA)']

##
plot(w_valid,  i_valid, 'r')
f= i.sub()[:,0,0].plot(fmt='g-', vmax=0.01)
figure()
df=i.sub()[:,0,0]- i_valid
dff=df/i_valid*100
dff.plot(fmt='k-', vmax=5, vmin=-5)
##
ff=figure()
ff.set_size_inches(44,6)
plot(w_valid,  ip_valid, 'r')
f= ip.sub()[:,0,0].plot(fmt='.g', vmax=0.005)
ff.savefig('/home/did/RTC/SMART-G/validation/alis_fts.png',dpi=300)
ff=figure()
ff.set_size_inches(44,6)
df=ip.sub()[:,0,0]- ip_valid
dff=df/ip_valid*100
dff.plot(fmt='.k-', vmax=5, vmin=-5)
ff.savefig('/home/did/RTC/SMART-G/validation/alis_fts_dif.png',dpi=300)

## CO2

In [ ]:
S=Smartg(alis={'nlow':3}, double=True)

In [ ]:
fgas = '/home/mathieu/work/MCCuda/artdeco_output/cTauGas_ray_glitter_CO2.dat'
gas_valid = np.loadtxt(fgas, skiprows=7)[:,1:].T
z_valid   = np.loadtxt(fgas, skiprows=7)[:,0]
w_valid   = np.array(open(fgas).readlines()[5].split()).astype(float)
fray = '/home/mathieu/work/MCCuda/artdeco_output/old/Emde_like/cTauRay_ray_glitter_CO2.dat'
ray_valid = np.loadtxt(fray, skiprows=7)[:,1:].T
data_valid=np.loadtxt('/home/mathieu/work/MCCuda/resultat/artdeco_lbl_nstr_32_ray_glitter_CO2.dat')

In [ ]:
gg=LUT(diff1(gas_valid,axis=1))
atm_valid = AtmAFGL('afglmw', grid=z_valid, O3=0., NO2=False, pfwav=[760.], prof_ray= ray_valid, 
                    prof_abs=gg)

In [ ]:
##%prun  
le={}
le.update(th=np.array([20.])*np.pi/180)
le.update(phi=np.array([180.])*np.pi/180)
surf=RoughSurface(SUR=1)
mle_valid_co2=  S.run(THVDEG=30., wl=w_valid, le=le,  DEPO=0., NBPHOTONS=1e8, 
                       surf=surf, atm=atm_valid, NBLOOP=10000)
print mle_valid_co2.attrs['kernel time (s)']

In [ ]:
i_valid=data_valid[:,1]
ip_valid=np.sqrt(data_valid[:,2]*data_valid[:,2]+data_valid[:,3]*data_valid[:,3])
ip=mle_valid_co2['Q_up (TOA)']*mle_valid_co2['Q_up (TOA)']+mle_valid_co2['U_up (TOA)']*mle_valid_co2['U_up (TOA)']
ip=ip.apply(np.sqrt,mdesc('Ip_up (TOA)'))

i=mle_valid_co2['I_up (TOA)']
i.desc=mdesc('I_up (TOA)')
##
plot(w_valid,  i_valid, 'r')
f= i.sub()[:,0,0].plot(fmt='g-', vmax=0.2)
figure()
df=i.sub()[:,0,0]- i_valid
dff=df/i_valid*100
dff.plot(fmt='k-', vmax=.3, vmin=-.3)
##
figure()
plot(w_valid,  ip_valid, 'r')
f= ip.sub()[:,0,0].plot(fmt='g-', vmax=0.1, vmin=0)
figure()
df=ip.sub()[:,0,0]- ip_valid
dff=df/ip_valid*100
dff.plot(fmt='k-', vmax=.3, vmin=-.3)